In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [2]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [3]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')
loansContract = web3.eth.contract(address=loansContractAddress, abi=loansContractABI)


In [4]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0xfbef6a46f5046196bd5324654b6b8503c84ce4d0443068ece8a289297b2c25d9')

In [5]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xb5f53d15806fb6938ea6061005f904d48b3ca7773a684f13cd24748bdbf2e18c')

In [6]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount 
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [7]:
deleteUser()

In [8]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [26]:
# Suppose bank wants to create a loan
account = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(account).call()

Enter the addrses: 0x87ef3D53218e6DF86a4C8EF2ef2356dd1D824Fd7


In [27]:
index

1

## Get pending loans

In [11]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [12]:
# Get pending loans using just one function
# Compiling Loans.sol contract with pragma experimental ABIEncoderV2;
def getPendingLoans2(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getLoans().processReceipt(receipt)
    print(rich_logs)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values
def getPendingLoansList2(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = []
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            pendingLoans.append(getPendingLoans2(_userContract, _loanieAddress, privateKey))
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [12]:
#getPendingLoansList2(index, userContract, accountsConract, account)

In [17]:
getPendingLoansList(index, userContract, accountsConract, account)

Enter your password: ········


['16914 0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C 1589380174 ']

## Confirm or reject loans

In [28]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(index, userContract, accountsConract, account)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for address: ' + pendingLoan.split(' ')[1])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': account,
        'nonce': web3.eth.getTransactionCount(account)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': account,
        'nonce': web3.eth.getTransactionCount(account)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for address: 0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C
c/r?c
for address: 0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C
c/r?c
for address: 0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C
c/r?r
for address: 0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C
c/r?r


# Create a new loan

In [13]:
account

'0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C'

In [25]:
# Create a new loan
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if  loanieType:
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=account, _amount=random.randint(1,100000), _organizationContract=organizationContract):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Enter the password of the organization: ········
Loan created


In [29]:
loans=loansContract.functions.getLoans().call()
print(loans)

[(1589380174, '0x87ef3D53218e6DF86a4C8EF2ef2356dd1D824Fd7', '0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C', 16914), (1589380452, '0x87ef3D53218e6DF86a4C8EF2ef2356dd1D824Fd7', '0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C', 62265), (1589380463, '0x87ef3D53218e6DF86a4C8EF2ef2356dd1D824Fd7', '0xfe86f9DF56920521188Dc9FD90E8f4299e67E33C', 37001)]
